In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [74]:
SPARCS_df = pd.read_csv('./Hospital_Inpatient_Discharges__SPARCS_De-Identified___Cost_Transparency__Beginning_2009_20250419_original.csv')
SPARCS_df.isna().sum()

Year                                     0
Facility Id                              0
Facility Name                            0
APR DRG Code                             0
APR Severity of Illness Code             0
APR DRG Description                      0
APR Severity of Illness Description    210
APR Medical Surgical Code              479
APR Medical Surgical Description         0
Discharges                               0
Mean Charge                              0
Median Charge                            0
Mean Cost                                0
Median Cost                              0
dtype: int64

In [75]:
len(SPARCS_df)

1192827

## APR Severity of Illness Code

In [76]:
SPARCS_df['APR Severity of Illness Code'].unique()

array([1, 2, 3, 4, 0], dtype=int64)

In [77]:
SPARCS_df['APR Severity of Illness Description'].unique()

array(['Minor', 'Moderate', 'Major', 'Extreme', 'Not Applicable', nan],
      dtype=object)

In [78]:
SPARCS_df[
    (SPARCS_df['APR Severity of Illness Description'] == 'Not Applicable') |
    (SPARCS_df['APR Severity of Illness Description'].isnull())
]

,Year,Facility Id,Facility Name,APR DRG Code,APR Severity of Illness Code,APR DRG Description,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Median Charge,Mean Cost,Median Cost
1523,2016,1,Albany Medical Center Hospital,956,0,Ungroupable,Not Applicable,NaN,Other,1,31000.32,31000.32,12274.23,12274.23
3546,2016,85,Auburn Community Hospital,955,0,Principal Diagnosis Invalid As Discharge Diagn...,Not Applicable,NaN,Other,1,13675.50,13675.50,4807.53,4807.53
4869,2016,1438,Bellevue Hospital Center,955,0,Principal Diagnosis Invalid As Discharge Diagn...,Not Applicable,NaN,Other,1,2545.00,2545.00,1322.79,1322.79
6362,2016,1178,Bronx-Lebanon Hospital Center - Concourse Divi...,955,0,Principal Diagnosis Invalid As Discharge Diagn...,Not Applicable,NaN,Other,10,7348.20,4676.48,6644.60,3976.82
7271,2016,1286,Brookdale Hospital Medical Center,955,0,Principal Diagnosis Invalid As Discharge Diagn...,Not Applicable,NaN,Other,1,16986.42,16986.42,9718.78,9718.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191076,2021,3067,Millard Fillmore Suburban Hospital,955,0,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,Not Applicable,NaN,Other,4,5647.67,4723.04,2922.06,1839.80
1191841,2021,3376,Cohen Children's Medical Center,955,0,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,Not Applicable,NaN,Other,1,60927.00,60927.00,8077.91,8077.91
1191842,2021,3376,Cohen Children's Medical Center,956,0,UNGROUPABLE,Not Applicable,NaN,Other,8,16277.62,14935.96,2908.46,2524.23
1192563,2021,3975,New York Presbyterian Hospital - Allen Hospital,955,0,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,Not Applicable,NaN,Other,4,36222.24,43319.90,11303.77,13371.07


In [79]:
SPARCS_df[
    (SPARCS_df['APR Severity of Illness Description'] == 'Not Applicable') |
    (SPARCS_df['APR Severity of Illness Description'].isnull())
]['APR DRG Description'].unique()

array(['Ungroupable',
       'Principal Diagnosis Invalid As Discharge Diagnosis',
       'Principal diagnosis invalid as discharge diagnosis',
       'PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGNOSIS',
       'UNGROUPABLE'], dtype=object)

In [80]:
SPARCS_df[
    (SPARCS_df['APR Severity of Illness Description'] == 'Not Applicable') 
]['APR Medical Surgical Code'].unique()

array([nan, 'U'], dtype=object)

In this situation I am thinking it wouldn't make sense to impute the missing or 'Not Applicable' values for Illness Description because then we would be teaching the model something that isn't true. And this can actually be a good thing because if the LLM parsing the user's natural language input cannot figure out what counts as 'Minor', 'Moderate', 'Major', 'Extreme' it can have 'Not Applicable' to fall back on and still be able to give the user an accurate prediction of what to expect.

In [81]:
SPARCS_df['APR Severity of Illness Description'] = SPARCS_df['APR Severity of Illness Description'].fillna('Not Applicable')

## Medical Surgical Code

APR Medical Surgical Code: The APR-DRG code indicating ‘M’ (Medical), ‘P’ (Surgical) or ‘NA’ (Other).

In [82]:
SPARCS_df['APR Medical Surgical Code'].unique()

array(['M', 'P', nan, 'U'], dtype=object)

In [83]:
SPARCS_df['APR Medical Surgical Description'].unique()

array(['Medical', 'Surgical', 'Other'], dtype=object)

Let's see if the description is anything besides 'Other' whenever the Surgical Code is missing. In that case we could calculate the true value by just mapping.

In [84]:
SPARCS_df[SPARCS_df['APR Medical Surgical Code'].isna()]['APR Medical Surgical Description'].unique()

array(['Other'], dtype=object)

In this case again I belive its better to just leave the nan and U(unkown/other) categories be.

In [85]:
SPARCS_df['APR Medical Surgical Code'] = SPARCS_df['APR Medical Surgical Code'].fillna('U')

Sanity Check

In [86]:
SPARCS_df.isna().sum()

Year                                   0
Facility Id                            0
Facility Name                          0
APR DRG Code                           0
APR Severity of Illness Code           0
APR DRG Description                    0
APR Severity of Illness Description    0
APR Medical Surgical Code              0
APR Medical Surgical Description       0
Discharges                             0
Mean Charge                            0
Median Charge                          0
Mean Cost                              0
Median Cost                            0
dtype: int64

Save the processed panda as a CSV

In [87]:
SPARCS_df.to_csv('./Hospital_Inpatient_Discharges__SPARCS_De-Identified___Cost_Transparency__Beginning_2009_20250419.csv', index=False)